In [1]:
import pandas as pd
import numpy as np

In [2]:
HOBO_low = pd.read_csv("HOBO\Group3-Low.csv", encoding="windows-1252")
HOBO_mid = pd.read_csv("HOBO\Group3-Mid.csv", encoding="windows-1252")
HOBO_top = pd.read_csv("HOBO\Group3-Top.csv", encoding="windows-1252")
CUBE = CUBE = pd.read_excel("CUBE\Group 3-Cube.xlsx")
results_1A = pd.read_csv("results_TOP.csv")

In [ ]:
v_a_low = HOBO_low.iloc[:, 3]
v_a_mid = HOBO_mid.iloc[:, 3]
v_a_top = HOBO_top.iloc[:, 3]
t_mean_op = results_1A.iloc[:,6]

7201
0       0.028590
1       0.000239
2       0.000162
3       0.000162
4       0.000162
          ...   
7196    0.091410
7197    0.138889
7198    0.135120
7199    0.112538
7200    0.063934
Name: Air_speed (m/s), Length: 7201, dtype: float64


In [110]:
def draught_rate_comfort(v_ave, t_op, interval):

    DR_standards = np.zeros((len(v_ave), 3))
    cols = ["Draught Rate", "ASHRAE Standard", "ISO 7730 Standard"]
    DR_standards = pd.DataFrame(DR_standards, columns=cols)

    for i in range(0, len(v_ave-interval), interval):
        v_sum_squared = 0
        v_rms = 0
        for j in range(interval):
            if i+j == len(v_ave-interval)-1:
                v = max(v_ave[i+j], 0.05)
                v_sum_squared += v*v
                v_rms = ((1/(j+1))*v_sum_squared)**0.5
                break
            v = max(v_ave[i+j], 0.05)
            v_sum_squared += v*v
        v_rms = ((1/interval)*v_sum_squared)**0.5
    
        for j in range(interval):
            if i+j == len(v_ave-interval)-1:
                v = max(v_ave[i+j], 0.05)
                TU = v_rms/v_ave[i+j]
                DR_standards.iloc[i+j,0] = (34 - t_op[i+j])*((v-0.05)**0.62)*(0.37*v*TU+3.14) 
                break
            v = max(v_ave[i+j], 0.05)
            TU = v_rms/v_ave[i+j]
            DR_standards.iloc[i+j,0] = (34 - t_op[i+j])*((v-0.05)**0.62)*(0.37*v*TU+3.14)

    for k in range(len(v_ave)):
        if DR_standards.iloc[k,0] <= 10:
            DR_standards.iloc[k,2] = "Cat I"
        elif DR_standards.iloc[k,0] <= 20:
            DR_standards.iloc[k,2] = "Cat II"
        elif DR_standards.iloc[k,0] <= 30:
            DR_standards.iloc[k,2] = "Cat III"
        else:
            DR_standards.iloc[k,2] = "X"

    for k in range(len(v_ave)):
        if DR_standards.iloc[k,0] <= 20:
            DR_standards.iloc[k,1] = "ok"
        else:
            DR_standards.iloc[k,1] = "X"

    return DR_standards

In [112]:
dr_results = draught_rate_comfort(v_a_mid, t_mean_op, 60)
print(dr_results["ISO 7730 Standard"].drop_duplicates())


C:\Users\aarne\AppData\Local\Temp\ipykernel_23056\1337488408.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Cat I' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  DR_standards.iloc[k,2] = "Cat I"
C:\Users\aarne\AppData\Local\Temp\ipykernel_23056\1337488408.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'ok' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  DR_standards.iloc[k,1] = "ok"


0         Cat I
1009     Cat II
1164          X
1189    Cat III
Name: ISO 7730 Standard, dtype: object


In [5]:
Tpr_north = CUBE.iloc[:,4]
Tpr_south = CUBE.iloc[:,6]
Tpr_east = CUBE.iloc[:,5]
Tpr_west = CUBE.iloc[:,7]
Tpr_up = CUBE.iloc[:,3]
Tpr_down = CUBE.iloc[:,8]

In [6]:
def radiant_temperature_asymmetry_comfort(n, s, e, w, u, d):
    DT_ns = n - s
    DT_ew = e - w
    DT_ud = u - d
    
    ISO_7730_warm_ns = []
    ISO_7730_cold_ns = []
    ASHRAE_warm_ns = []
    ASHRAE_cold_ns = []

    ISO_7730_warm_ew = []
    ISO_7730_cold_ew = []
    ASHRAE_warm_ew = []
    ASHRAE_cold_ew = []

    ISO_7730_warm_ud = []
    ISO_7730_cold_ud = []
    ASHRAE_warm_ud = []
    ASHRAE_cold_ud = []

    for i in range(len(DT_ns)):
        if DT_ns[i] > -10:
            ISO_7730_cold_ns.append("Cat I, II")
            ASHRAE_cold_ns.append("OK")
        elif DT_ns[i] > -13:
            ISO_7730_cold_ns.append("Cat III")
        else:
            ISO_7730_cold_ns.append("X")
            ASHRAE_cold_ns.append("X")

        if DT_ns[i] < 23:
            ISO_7730_warm_ns.append("Cat I, II")
            ASHRAE_warm_ns.append("OK")
        elif DT_ns[i] < 35:
            ISO_7730_warm_ns.append("Cat III")
        else:
            ISO_7730_warm_ns.append("X")
            ASHRAE_warm_ns.append("X")

    # Looping through each element in DT_ew
    for i in range(len(DT_ew)):  
        if DT_ew[i] > -10:
            ISO_7730_cold_ew.append("Cat I, II")
            ASHRAE_cold_ew.append("OK")
        elif DT_ew[i] > -13:
            ISO_7730_cold_ew.append("Cat III")
        else:
            ISO_7730_cold_ew.append("X")
            ASHRAE_cold_ew.append("X")

        if DT_ew[i] < 23:
            ISO_7730_warm_ew.append("Cat I, II")
            ASHRAE_warm_ew.append("OK")
        elif DT_ew[i] < 35:
            ISO_7730_warm_ew.append("Cat III")
        else:
            ISO_7730_warm_ew.append("X")
            ASHRAE_warm_ew.append("X")

    # Looping through each element in DT_ud
    for i in range(len(DT_ud)):  
        if DT_ud[i] > -14:
            ISO_7730_cold_ud.append("Cat I, II")
            ASHRAE_cold_ud.append("OK")
        elif DT_ud[i] > -18:
            ISO_7730_cold_ud.append("Cat III")
        else:
            ISO_7730_cold_ud.append("X")
            ASHRAE_cold_ud.append("X")

        if DT_ud[i] < 5:
            ISO_7730_warm_ud.append("Cat I, II")
            ASHRAE_warm_ud.append("OK")
        elif DT_ud[i] < 7:
            ISO_7730_warm_ud.append("Cat III")
        else:
            ISO_7730_warm_ud.append("X")
            ASHRAE_warm_ud.append("X")

    result = np.column_stack([DT_ns, ISO_7730_warm_ns, ISO_7730_cold_ns, ASHRAE_warm_ns, ASHRAE_cold_ns,
              DT_ew, ISO_7730_warm_ew, ISO_7730_cold_ew, ASHRAE_warm_ew, ASHRAE_cold_ew,
              DT_ud, ISO_7730_warm_ud, ISO_7730_cold_ud, ASHRAE_warm_ud, ASHRAE_cold_ud])
    
    cols = [
    "DT_ns", "ISO_7730_warm_ns", "ISO_7730_cold_ns", "ASHRAE_warm_ns", "ASHRAE_cold_ns",
    "DT_ew", "ISO_7730_warm_ew", "ISO_7730_cold_ew", "ASHRAE_warm_ew", "ASHRAE_cold_ew",
    "DT_ud", "ISO_7730_warm_ud", "ISO_7730_cold_ud", "ASHRAE_warm_ud", "ASHRAE_cold_ud"
    ]

    df = pd.DataFrame(result, columns=cols)
    
    return df



In [7]:
s = radiant_temperature_asymmetry_comfort(Tpr_north, Tpr_south, Tpr_east, Tpr_west, Tpr_up, Tpr_down)
print(s.head)

<bound method NDFrame.head of                      DT_ns ISO_7730_warm_ns ISO_7730_cold_ns ASHRAE_warm_ns  \
0     -0.18999999999999773        Cat I, II        Cat I, II             OK   
1     -0.15000000000000213        Cat I, II        Cat I, II             OK   
2      -0.0799999999999983        Cat I, II        Cat I, II             OK   
3      -0.0799999999999983        Cat I, II        Cat I, II             OK   
4     -0.10000000000000142        Cat I, II        Cat I, II             OK   
...                    ...              ...              ...            ...   
5368                 -2.16        Cat I, II        Cat I, II             OK   
5369   -2.1400000000000006        Cat I, II        Cat I, II             OK   
5370   -2.3200000000000003        Cat I, II        Cat I, II             OK   
5371    -2.349999999999998        Cat I, II        Cat I, II             OK   
5372   -2.3599999999999994        Cat I, II        Cat I, II             OK   

     ASHRAE_cold_ns  